# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
loadCSV = "../output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
cityData = pd.read_csv(loadCSV)
del cityData["City_ID"]
cityData

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Gmaps configure with API key
gmaps.configure(api_key=g_key)

In [81]:
#locations
locations = cityData[["Lat", "Lng"]].astype(float)

#humidity
humidity = cityData["Humidity"].astype(float)


fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=300,
                                   point_radius=5)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [79]:

#clean_city_data = cityDataCold.drop(toWindy, inplace=False)
clean_city_data = cityData[(cityData["Max Temp"] >70)&(cityData["Max Temp"]<80)&
                          (cityData["Wind Speed"]<10)]
clean_city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
44,puerto ayora,26,EC,1558378762,86,-0.74,-90.35,79.00,5.99
59,bontang,54,ID,1558378765,99,0.12,117.47,74.22,2.62


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
hotel_df = pd.DataFrame(clean_city_data, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,
12,avarua,CK,-21.21,-159.78,
25,kantang,TH,7.42,99.52,
44,puerto ayora,EC,-0.74,-90.35,
59,bontang,ID,0.12,117.47,


In [83]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [84]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))